In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

import os
from dotenv import load_dotenv
import boto3
import io

Data Loading

In [5]:
def load_s3_instagram_data():

    load_dotenv()
    aws_access_key = os.getenv("aws_accessKey")
    aws_secret_key = os.getenv("aws_secretKey")
    region_name='ap-northeast-2'

    # S3 클라이언트 생성
    client = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=region_name)

    today = datetime.now()
    year, week, _ = today.isocalendar()

    bucket_name = 'flexmatch-data'

    table_list = ['RECENT_USER_INFO_MTR', 'TIME_SERIES_PROFILE_INFO', 'BY_USER_ID_MEDIA_DTL_INFO', 'BY_DATE_MEDIA_AGG_INFO']
    df_by_table_list = {}

    for table_name in table_list:
        prefix = f'instagram-data/tables/{table_name}/year={year}/week=23/' # 이번주가 없어서 week를 따로 x
        response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

        if 'Contents' in response:
            files = [content['Key'] for content in response['Contents'] if content['Key'].endswith('merged_data.parquet')]

            if not files:
                print(f"[Info] No merged_data.parquet found for table: {table_name}")
                continue

            # 각 파일 순회
            for file_key in files:
                try:
                    obj = client.get_object(Bucket=bucket_name, Key=file_key)
                    df = pd.read_parquet(io.BytesIO(obj['Body'].read()))
                    df_by_table_list[table_name] = df
                    print(f"[Success] Loaded {file_key} for table {table_name}")
                except Exception as e:
                    print(f"[Error] Failed to read {file_key} for table {table_name}: {e}")
        else:
            print(f"[Info] No files found under prefix: {prefix}")
        
    return df_by_table_list

df_by_table_list = load_s3_instagram_data()

# recent_user_info_mtr_2 = df_by_table_list['RECENT_USER_INFO_MTR']
# time_series_profile_info_2 = df_by_table_list['TIME_SERIES_PROFILE_INFO']
# by_user_id_media_dtl_info_2 = df_by_table_list['BY_USER_ID_MEDIA_DTL_INFO']
# by_date_media_agg_info_2 = df_by_table_list['BY_DATE_MEDIA_AGG_INFO']

recent_user_info_mtr = df_by_table_list['RECENT_USER_INFO_MTR']
time_series_profile_info = df_by_table_list['TIME_SERIES_PROFILE_INFO']
by_user_id_media_dtl_info = df_by_table_list['BY_USER_ID_MEDIA_DTL_INFO']
by_date_media_agg_info = df_by_table_list['BY_DATE_MEDIA_AGG_INFO']



[Success] Loaded instagram-data/tables/RECENT_USER_INFO_MTR/year=2025/week=23/merged_data.parquet for table RECENT_USER_INFO_MTR
[Success] Loaded instagram-data/tables/TIME_SERIES_PROFILE_INFO/year=2025/week=23/merged_data.parquet for table TIME_SERIES_PROFILE_INFO
[Success] Loaded instagram-data/tables/BY_USER_ID_MEDIA_DTL_INFO/year=2025/week=23/merged_data.parquet for table BY_USER_ID_MEDIA_DTL_INFO
[Success] Loaded instagram-data/tables/BY_DATE_MEDIA_AGG_INFO/year=2025/week=23/merged_data.parquet for table BY_DATE_MEDIA_AGG_INFO


In [ ]:
# # 날짜까지 순회하게 만든 로직

# load_dotenv()
# aws_access_key = os.getenv("aws_accessKey")
# aws_secret_key = os.getenv("aws_secretKey")
# region_name = 'ap-northeast-2'

# client = boto3.client(
#     's3',
#     aws_access_key_id=aws_access_key,
#     aws_secret_access_key=aws_secret_key,
#     region_name=region_name
# )

# today = datetime.now()
# year, current_week, _ = today.isocalendar()

# last_week_date = today - timedelta(weeks=1)
# last_week_year, last_week, _ = last_week_date.isocalendar()

# week_year_pairs = [(year, current_week), (last_week_year, last_week)]

# bucket_name = 'flexmatch-data'
# table_list = ['RECENT_USER_INFO_MTR', 'TIME_SERIES_PROFILE_INFO', 'BY_USER_ID_MEDIA_DTL_INFO', 'BY_DATE_MEDIA_AGG_INFO']

# merged_data_by_table = {table_name: {} for table_name in table_list}

# # 주차별 순회
# for year_val, week_val in week_year_pairs:
#     for table_name in table_list:
#         prefix = f'instagram-data/tables/{table_name}/year={year_val}/week={week_val}/'
#         response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

#         if 'Contents' in response:
#             target_files = [
#                 content['Key']
#                 for content in response['Contents']
#                 if content['Key'].endswith('merged_data.parquet')
#             ]

#             if not target_files:
#                 print(f"[Info] No merged_data.parquet found for {table_name} week={week_val}")
#                 continue

#             for file_key in target_files:
#                 try:
#                     obj = client.get_object(Bucket=bucket_name, Key=file_key)
#                     df = pd.read_parquet(io.BytesIO(obj['Body'].read()))
#                     merged_data_by_table[table_name][week_val] = df
#                     print(f"[Success] Loaded {file_key} for table {table_name}, week {week_val}")
#                 except Exception as e:
#                     print(f"[Error] Failed to read {file_key} for {table_name}, week {week_val}: {e}")
#         else:
#             print(f"[Info] No files found under prefix: {prefix}")


basic preprocessing

In [6]:
recent_user_info_mtr = recent_user_info_mtr.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
time_series_profile_info = time_series_profile_info.drop_duplicates(subset=['base_ymd', 'acnt_id'])
by_user_id_media_dtl_info = by_user_id_media_dtl_info.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
by_date_media_agg_info = by_date_media_agg_info.drop_duplicates(subset=['base_ymd', 'media_id'])

recent_user_info_mtr_2 = recent_user_info_mtr_2.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
time_series_profile_info_2 = time_series_profile_info_2.drop_duplicates(subset=['base_ymd', 'acnt_id'])
by_user_id_media_dtl_info_2 = by_user_id_media_dtl_info_2.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
by_date_media_agg_info_2 = by_date_media_agg_info_2.drop_duplicates(subset=['base_ymd', 'media_id'])


가장 최근 데이터로 isin 해줘야 할 것 같음. 그래야 1주일 전, 후를 비교할 수 있기 때문임

In [171]:
# # 각 acnt_id가 있는 전부 있는 경우만 가능, isin으로 제거
# # 근데 전 주차에는 없는 유저가 생길수도 있음
# # media 같은 경우에도 최근 25개만 일단 가져오고 있기 때문에 일주일 단위여도 없을수도 있음

# # 이렇게 해야하는 이유는 timeseries 변수를 계산하려면 무조건 두 주차에 같은 아이디가 있어야 함
# # 또한, user info랑 media info 같이 써야하는 경우에는 어짜피 한번 거르고 있을 듯
# # -> 따라서, 굳이 필수적으로 하지는 않아도 될듯함

# unique_user = recent_user_info_mtr['acnt_id'].unique()

# time_series_profile_info = time_series_profile_info[time_series_profile_info['acnt_id'].isin(unique_user)]
# by_user_id_media_dtl_info = by_user_id_media_dtl_info[by_user_id_media_dtl_info['acnt_id'].isin(unique_user)]

# unique_media = by_user_id_media_dtl_info['media_id'].unique()
# by_date_media_agg_info = by_date_media_agg_info[by_date_media_agg_info['media_id'].isin(unique_media)]


In [7]:
def influencer_scale_type(row):
    count = row['follower_cnt']
    if count < 1000:
        return 'nano'
    elif 1000 <= count <= 10000:
        return 'micro'
    elif 10000 < count <= 100000:
        return 'mid'
    elif 100000 < count <= 500000:
        return 'macro'
    else:
        return 'mega'

recent_user_info_mtr['influencer_scale_type'] = recent_user_info_mtr.apply(influencer_scale_type, axis=1)
recent_user_info_mtr_2['influencer_scale_type'] = recent_user_info_mtr_2.apply(influencer_scale_type, axis=1)


C:\Users\ehddl\AppData\Local\Temp\ipykernel_17764\3577675420.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_user_info_mtr['influencer_scale_type'] = recent_user_info_mtr.apply(influencer_scale_type, axis=1)


create merged file

In [ ]:
# all_merged_df_a = pd.merge(recent_user_info_mtr, time_series_profile_info, on='acnt_id', how='left')
# all_merged_df_b = pd.merge(all_merged_df_a, by_user_id_media_dtl_info, on='acnt_id')

# all_merged_df = pd.merge(all_merged_df_b, by_date_media_agg_info, on='media_id')

# media_engagement_merged_df = pd.merge(by_user_id_media_dtl_info, by_date_media_agg_info, on='media_id')
# media_engagement_profile_merged_df = pd.merge(media_engagement_merged_df, recent_user_info_mtr, on='acnt_id', how='left')
# time_series_merged_df = pd.merge(time_series_profile_info, time_series_profile_info_2, on='acnt_id')

In [9]:
def create_merged_df(user_info_df, timeseries_df, timeseries_df_2, media_info_df, media_agg_df):
    all_merged_df_a = pd.merge(user_info_df, timeseries_df, on='acnt_id', how='left')
    all_merged_df_b = pd.merge(all_merged_df_a, media_info_df, on='acnt_id')
    all_merged_df = pd.merge(all_merged_df_b, media_agg_df, on='media_id')

    media_engagement_merged_df = pd.merge(media_info_df, media_agg_df, on='media_id')

    # 절반 이상의 게시물의 like가 비공개인 influencer 제거
    by_user_na_like_count = media_engagement_merged_df[media_engagement_merged_df['like_cnt'].isna()].groupby(['acnt_id'])['media_id'].count()
    na_like_user = by_user_na_like_count[by_user_na_like_count > 13].index
    media_engagement_merged_df = media_engagement_merged_df[~media_engagement_merged_df['acnt_id'].isin(na_like_user)].reset_index()

    # fillna 함수 정의
    like_medians = media_engagement_merged_df.groupby(['acnt_id'])['like_cnt'].median()
    comments_medians = media_engagement_merged_df.groupby(['acnt_id'])['cmnt_cnt'].median()

    def fillna_like(row):
        if pd.notna(row['like_cnt']):
            return row['like_cnt']
        median = like_medians.get(row['acnt_id'])
        return median if pd.notna(median) else 0

    def fillna_comment(row):
        if pd.notna(row['cmnt_cnt']):
            return row['cmnt_cnt']
        median = comments_medians.get(row['acnt_id'])
        return median if pd.notna(median) else 0

    media_engagement_merged_df['like_cnt'] = media_engagement_merged_df.apply(fillna_like, axis=1)
    media_engagement_merged_df['cmnt_cnt'] = media_engagement_merged_df.apply(fillna_comment, axis=1)

    
    media_engagement_profile_merged_df = pd.merge(media_engagement_merged_df, user_info_df, on='acnt_id', how='left')
    time_series_merged_df = pd.merge(timeseries_df, timeseries_df_2, on='acnt_id')

    return all_merged_df, media_engagement_merged_df, media_engagement_profile_merged_df, time_series_merged_df

In [10]:
all_merged_df, media_engagement_merged_df, media_engagement_profile_merged_df, time_series_merged_df = create_merged_df(
                                                                                                                        recent_user_info_mtr_2,
                                                                                                                        time_series_profile_info,
                                                                                                                        time_series_profile_info_2,
                                                                                                                        by_user_id_media_dtl_info_2,
                                                                                                                        by_date_media_agg_info_2)

크리에이터 활동성

- 실제 게시물 업로드 날짜를 기준으로 계산할 수 있을 것 같음
- 최근 25개의 게시물의 평균 게시 간격을 계산
- 최근 25개의 게시물을 며칠에 걸쳐서 업로드 했는지에 대한 값을 계산

In [ ]:
media_dtl_2_copy = by_user_id_media_dtl_info_2.copy()
media_dtl_2_copy = media_dtl_2_copy.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
media_dtl_2_copy['reg_dt'] = pd.to_datetime(media_dtl_2_copy['reg_dt'])
media_dtl_2_copy = media_dtl_2_copy.sort_values(['acnt_id', 'reg_dt'])

# 게시물 간격 계산
media_dtl_2_copy['prev_reg_dt'] = media_dtl_2_copy.groupby('acnt_id')['reg_dt'].shift(1)
media_dtl_2_copy['gap_days'] = (media_dtl_2_copy['reg_dt'] - media_dtl_2_copy['prev_reg_dt']).dt.days
# media_dtl_2_copy

# gap_days가 NaN인 첫 번째 포스트 제외 후 평균 간격 계산
activity_df = media_dtl_2_copy.dropna(subset=['gap_days']).groupby('acnt_id')['gap_days'].mean().reset_index()
activity_df.rename(columns={'gap_days': 'avg_upload_interval'}, inplace=True)

# 활동성 점수 계산 (간격의 역수로 환산) -> 점수 정규화 (업로드 간격이 짧을수록 점수가 높아지도록 역수를 취해서 계산한 것)
activity_df['activity_score'] = 100 / activity_df['avg_upload_interval']
activity_df

,media_id,acnt_id,media_type_nm,reels_feed_type_nm,media_url_addr,media_unq_url_addr,tmnl_url_addr,reg_dt,media_cn,acnt_conn_yn,feed_share_yn,cmnt_actvtn_yn,prev_reg_dt,gap_days
1931,17909318463036853,17841400002751973,VIDEO,REELS,https://scontent-nrt1-2.cdninstagram.com/o1/v/...,https://www.instagram.com/reel/DKE4nvPTnMu/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,2025-05-25 12:20:56+00:00,📎키즈구매시 퓨어증정이벤트는 종료 되었습니다\n\n오메가3 마감임박입니다♥️,N,None,None,NaT,NaN
18512,18040241717290389,17841400002751973,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DKE8TbfTxig/,None,2025-05-25 12:52:32+00:00,",\n모로는 모참쮜~🍊\n다음 공구일이 3개월 이후니까 쟁여주십시오…🫠\n\n하루 ...",N,None,None,2025-05-25 12:20:56+00:00,0.0
6755,17971024994896809,17841400002751973,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/DKFERm8zmgG/,None,2025-05-25 14:02:12+00:00,",\n🔸받은 제품은 햇빛이 안드는 서늘한 곳 에 보관하시고 오픈후엔 냉장보관 및 9...",N,None,None,2025-05-25 12:52:32+00:00,0.0
23520,17893733829236933,17841400002751973,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/DKFE8ANzG6t/,None,2025-05-25 14:07:59+00:00,",\n카디이/키즈 구매시 퓨어🟡 증정 됩니다.\n\n레몬향, 레몬맛, 달달쓰\n비타...",N,None,None,2025-05-25 14:02:12+00:00,0.0
18511,18031270604413907,17841400002751973,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/DKGOi45T7hc/,None,2025-05-26 00:51:11+00:00,",\n#현재 키즈주문 많아요🐟🩵\n#오메가3#키즈오메가3#에스키모오메가3#에스키모오...",N,None,None,2025-05-25 14:07:59+00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12535,18081083746755990,17841474495202295,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/DJoxxQ5RhCt/,None,2025-05-14 14:21:45+00:00,susu_23.06.30 #꾹꾹이 #슈 #아프지말자\n\n나는 꾹꾹이 하다가 잠드는...,N,None,None,2025-05-14 12:44:18+00:00,0.0
23517,18042903941207161,17841474495202295,VIDEO,REELS,https://scontent-nrt1-2.cdninstagram.com/o1/v/...,https://www.instagram.com/reel/DJpb5R0TJCa/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,2025-05-14 20:32:05+00:00,#넷플릭스정주행 #산책후 #반려견과함께 #멍마들,N,None,None,2025-05-14 14:21:45+00:00,0.0
12534,18073647475924861,17841474495202295,VIDEO,REELS,https://scontent-nrt1-2.cdninstagram.com/o1/v/...,https://www.instagram.com/reel/DJv-FeQT7aU/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,2025-05-17 09:25:59+00:00,우비 산 기념으로 나갔다가 2분만에 들어오는거 \n어찌할꼬....ㅠㅠ🐶😅😅,N,None,None,2025-05-14 20:32:05+00:00,2.0
16933,18134901241417728,17841474495202295,VIDEO,REELS,https://scontent-nrt1-2.cdninstagram.com/o1/v/...,https://www.instagram.com/reel/DKPO8Fqx1M2/,https://scontent-nrt1-2.cdninstagram.com/v/t51...,2025-05-29 12:50:22+00:00,#멍라운지24 #반려견 #케어 #전문점 \n#교육 #호텔링 #아프지말자,N,None,None,2025-05-17 09:25:59+00:00,12.0


In [12]:
def calculate_activity_score(recent_media_dtl_df): # 두 개의 테이블 중 가장 최근
    media_dtl_copy = recent_media_dtl_df.copy()
    media_dtl_copy = media_dtl_copy.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
    media_dtl_copy['reg_dt'] = pd.to_datetime(media_dtl_copy['reg_dt'])
    media_dtl_copy = media_dtl_copy.sort_values(['acnt_id', 'reg_dt'])

    # 게시물 간격 계산
    media_dtl_copy['prev_reg_dt'] = media_dtl_copy.groupby('acnt_id')['reg_dt'].shift(1)
    media_dtl_copy['gap_days'] = (media_dtl_copy['reg_dt'] - media_dtl_copy['prev_reg_dt']).dt.days

    # gap_days가 NaN인 첫 번째 포스트 제외 후 평균 간격 계산
    activity_df = media_dtl_copy.dropna(subset=['gap_days']).groupby('acnt_id')['gap_days'].mean().reset_index()
    activity_df.rename(columns={'gap_days': 'avg_upload_interval'}, inplace=True)

    # 활동성 점수 계산 (간격의 역수로 환산) -> 점수 정규화 (업로드 간격이 짧을수록 점수가 높아지도록 역수를 취해서 계산한 것)
    activity_df['activity_score'] = 100 / activity_df['avg_upload_interval']
    
    return activity_df

In [13]:
activity_df = calculate_activity_score(by_user_id_media_dtl_info_2)

In [71]:
activity_df

,acnt_id,avg_upload_interval,activity_score
0,17841400002751973,0.333333,300.000000
1,17841400006499942,3.166667,31.578947
2,17841400011995621,2.291667,43.636364
3,17841400016295781,0.541667,184.615385
4,17841400025364370,26.416667,3.785489
...,...,...,...
726,17841473332885823,2.750000,36.363636
727,17841473347851502,2.208333,45.283019
728,17841473857435791,0.500000,200.000000
729,17841474152637635,3.500000,28.571429


In [ ]:
def check_inf(df):
    float_cols = df.select_dtypes(include=['float64', 'float32']).columns

    mask_inf = np.isinf(df[float_cols]).any(axis=1)
    mask_neginf = np.isneginf(df[float_cols]).any(axis=1)

    invalid_rows = df[mask_inf | mask_neginf]

    print(f"⚠️ inf / -inf 포함 행 개수: {len(invalid_rows)}개")
    display(invalid_rows)

In [52]:
check_inf(activity_df)

⚠️ inf / -inf 포함 행 개수: 10개


,acnt_id,avg_upload_interval,activity_score
16,17841400106478893,0.0,inf
49,17841400328187277,0.0,inf
87,17841400597819049,0.0,inf
190,17841401409523807,0.0,inf
243,17841401837605088,0.0,inf
284,17841402140800589,0.0,inf
384,17841405136373546,0.0,inf
393,17841405619465983,0.0,inf
404,17841406027893172,0.0,inf
443,17841407619873635,0.0,inf


트렌드지수 - 팔로워 순증감률 (follower_growth_rate)

In [126]:
# time_series_profile_info_2['acnt_id'] = time_series_profile_info_2['acnt_id'].astype(object)

# influencer_list = time_series_profile_info['acnt_id'].unique()
# time_series_profile_info_2[time_series_profile_info_2['acnt_id'].isin(influencer_list)]
# time_series_merged_df = pd.merge(time_series_profile_info, time_series_profile_info_2, on='acnt_id')

# time_series_merged_df['trend_score'] = ((time_series_merged_df['follower_cnt_y'] - time_series_merged_df['follower_cnt_x']) / (time_series_merged_df['follower_cnt_x'])) * 100
# time_series_merged_df[['acnt_id', 'trend_score']]

In [14]:
def calculate_follower_growth_rate(time_series_df, recent_time_series_df):
    time_series_df.loc[:, 'acnt_id'] = time_series_df['acnt_id'].astype(object)
    recent_time_series_df.loc[:, 'acnt_id'] = recent_time_series_df['acnt_id'].astype(object)

    influencer_list = time_series_df['acnt_id'].unique()
    recent_time_series_df = recent_time_series_df[recent_time_series_df['acnt_id'].isin(influencer_list)]
    time_series_merged_df = pd.merge(time_series_df, recent_time_series_df, on='acnt_id')

    time_series_merged_df['trend_score'] = ((time_series_merged_df['follower_cnt_y'] - time_series_merged_df['follower_cnt_x']) / (time_series_merged_df['follower_cnt_x'])) * 100
    growth_rate_df = time_series_merged_df[['acnt_id', 'trend_score']]

    return growth_rate_df

In [15]:
growth_rate_df = calculate_follower_growth_rate(time_series_profile_info, time_series_profile_info_2)

##### 팔로워 참여도

- 26일 데이터를 기준으로 계산 진행
- 단일 프소트 기준이 아닌 전체 포스트 평균을 측정
- media_dtl & media_agg & user_data를 병합한 데이터

---------------------------------------------------------

- 좋아요 비공개인 경우에는 각각의 중앙값으로 대체 -> 문제는 아예 없는 사람은 어떻게 해야하는건지...
- estimated_total_engagement = avg_engagement_per_post * total_post_count
- avg_engagement_per_post = like+comments / 25
- total_post_count -> user_data

In [129]:
# media_engagement_merged_df = pd.merge(media_dtl, media_agg, on='media_id')


# # 해당 부분 수정 -> dic에 자동으로 median 값이 들어갈 수 있도록 수정
# media_engagement_merged_df.groupby(['acnt_id'])['like_cnt'].median()
# fillna_dict = {
#     '17841400591698216': 810,
#     '17841401506106699': 791,
#     '17841401839008777': 480,
#     '17841402936102997': 96,
#     '17841406083664639': 83,
#     '17841400361359004' : 0,
#     '17841401473518226' : 0
# }

# for acnt_id, default_like in fillna_dict.items():
#     condition = media_engagement_merged_df['acnt_id'] == acnt_id
#     media_engagement_merged_df.loc[condition, 'like_cnt'] = (
#         media_engagement_merged_df.loc[condition, 'like_cnt'].fillna(default_like)
#     )

# media_engagement_merged_df = pd.merge(media_engagement_merged_df, user_info_2,  on='acnt_id', how = 'left')
# media_engagement_merged_df_2 = media_engagement_merged_df[['acnt_id', 'media_id', 'follower_cnt', 'follow_cnt', 'like_cnt', 'cmnt_cnt', 'media_cnt']]

# engaged_df = media_engagement_merged_df_2.groupby(['acnt_id']).agg({
#     'like_cnt' : 'sum',
#     'cmnt_cnt' : 'sum',
#     'media_cnt': 'first',
#     'follower_cnt' : 'first',
#     'follow_cnt' : 'first'
# }).reset_index()

# engaged_df['avg_engagement_per_post'] = ((engaged_df['like_cnt'] + engaged_df['cmnt_cnt']) / 25)
# engaged_df['estimated_total_engagement'] = engaged_df['avg_engagement_per_post'] * engaged_df['media_cnt']
# engaged_df['follower_total_engagement'] = (engaged_df['estimated_total_engagement'] / engaged_df['follower_cnt']) * 100
# engaged_df

In [16]:
def calculate_follower_engagement(media_engagement_profile_merged_df):
    media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df[['acnt_id', 'media_id', 'follower_cnt', 'follow_cnt', 'like_cnt', 'cmnt_cnt', 'media_cnt']]
    
    engaged_df = media_engagement_profile_merged_df_copy.groupby(['acnt_id']).agg({
        'like_cnt' : 'sum',
        'cmnt_cnt' : 'sum',
        'media_cnt': 'first',
        'follower_cnt' : 'first',
        'follow_cnt' : 'first'
    }).reset_index()

    engaged_df['avg_engagement_per_post'] = ((engaged_df['like_cnt'] + engaged_df['cmnt_cnt']) / 25)
    engaged_df['estimated_total_engagement'] = engaged_df['avg_engagement_per_post'] * engaged_df['media_cnt']
    engaged_df['follower_total_engagement'] = (engaged_df['estimated_total_engagement'] / engaged_df['follower_cnt']) * 100
    
    follower_engagment_df = engaged_df

    return follower_engagment_df

In [17]:
follower_engagment_df = calculate_follower_engagement(media_engagement_profile_merged_df)

In [ ]:
check_inf(follower_engagment_df)

⚠️ inf / -inf 포함 행 개수: 1개


,acnt_id,like_cnt,cmnt_cnt,media_cnt,follower_cnt,follow_cnt,avg_engagement_per_post,estimated_total_engagement,follower_total_engagement
454,17841465816785419,1387.5,101,11,0,0,59.54,654.94,inf


In [ ]:
recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_id']=='17841465816785419']
recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_id']=='17841465816785419']

,acnt_id,acnt_nm,web_addr,acnt_sub_nm,intro_txt,profile_photo_url_addr,acnt_conn_yn,category_nm,follower_cnt,follow_cnt,media_cnt,influencer_scale_type
609,17841465816785419,kimchis.exe,https://litt.ly/kimchis,김치스(kimchi’s) | 김치 편집샵,"전국 빨간맛 편집샵🔥\n매콤하고, 맛있는 음식을 수집합니다.",https://scontent-nrt1-2.xx.fbcdn.net/v/t51.757...,N,홈/리빙,0,0,11,nano


##### 팔로워 충성도

- user_info or time_series 데이터가 필요
- 트렌드 지수는 time_series 데이터를 가지고 단순히 빼기를 진행했었음

In [ ]:
# time_series_merged_df_copy = time_series_merged_df[['acnt_id', 'follower_cnt_x', 'follower_cnt_y']]

# time_series_merged_df_copy['follower_change'] = (time_series_merged_df_copy['follower_cnt_y'] - time_series_merged_df_copy['follower_cnt_x'])

# def estimate_new_follower(row):
#     if row['follower_change'] < 0:
#         return 0
#     else:
#         return row['follower_change']

# time_series_merged_df_copy['new_follower'] = time_series_merged_df_copy.apply(estimate_new_follower, axis=1)

# # 충성도 계산
# # time_series_merged_df_copy = time_series_merged_time_series_merged_df_copy_copy.copy()
# time_series_merged_df_copy['unfollowed'] = time_series_merged_df_copy['follower_cnt_x'] + time_series_merged_df_copy['new_follower'] - time_series_merged_df_copy['follower_cnt_y']
# time_series_merged_df_copy['follower_retention_rate'] = ((time_series_merged_df_copy['follower_cnt_x'] - time_series_merged_df_copy['unfollowed']) / time_series_merged_df_copy['follower_cnt_x']) * 100
# time_series_merged_df_copy['follower_retention_rate'] = time_series_merged_df_copy['follower_retention_rate'].round(2)

# time_series_merged_df_copy

In [ ]:
# def estimate_loyalty(row):
#     if row['follower_change'] < 0:
#         return ((row['follower_cnt_x'] + row['follower_change']) / row['follower_cnt_x']) * 100
#     else:
#         return 100.0

# time_series_merged_df_copy['follower_loyalty'] = time_series_merged_df_copy.apply(estimate_loyalty, axis=1)
# time_series_merged_df_copy['follower_loyalty'] = time_series_merged_df_copy['follower_loyalty'].round(2)


In [18]:
def calculate_follower_loyalty(time_series_merged_df):
    time_series_merged_df_copy = time_series_merged_df[['acnt_id', 'follower_cnt_x', 'follower_cnt_y']].copy()

    time_series_merged_df_copy.loc[:, 'follower_change'] = (time_series_merged_df_copy['follower_cnt_y'] - time_series_merged_df_copy['follower_cnt_x'])

    def estimate_new_follower(row):
        if row['follower_change'] < 0:
            return 0
        else:
            return row['follower_change']

    time_series_merged_df_copy.loc[:, 'new_follower'] = time_series_merged_df_copy.apply(estimate_new_follower, axis=1)
    time_series_merged_df_copy.loc[:, 'unfollowed'] = time_series_merged_df_copy['follower_cnt_x'] + time_series_merged_df_copy['new_follower'] - time_series_merged_df_copy['follower_cnt_y']
    time_series_merged_df_copy.loc[:, 'follower_retention_rate'] = ((time_series_merged_df_copy['follower_cnt_x'] - time_series_merged_df_copy['unfollowed']) / time_series_merged_df_copy['follower_cnt_x']) * 100
    time_series_merged_df_copy.loc[:, 'follower_retention_rate'] = time_series_merged_df_copy['follower_retention_rate'].round(2)

    follower_loyalty_df = time_series_merged_df_copy

    return follower_loyalty_df

In [40]:
follower_loyalty_df = calculate_follower_loyalty(time_series_merged_df)
follower_loyalty_df

,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate
0,17841472095454621,2348,2424,76,76,0,100.00
1,17841459031207937,2119,2119,0,0,0,100.00
2,17841458105192949,1269,1271,2,2,0,100.00
3,17841400305959659,12337,12319,-18,0,18,99.85
4,17841401243247010,58980,58951,-29,0,29,99.95
...,...,...,...,...,...,...,...
718,17841403740294688,13469,13457,-12,0,12,99.91
719,17841405305728171,5791,5791,0,0,0,100.00
720,17841400214059012,112637,111412,-1225,0,1225,98.91
721,17841430071957252,1999,1997,-2,0,2,99.90


In [43]:
follower_loyalty_df[follower_loyalty_df['acnt_id']=='17841465816785419']

,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate
683,17841465816785419,51,0,-51,0,51,0.0


In [42]:
check_inf(follower_loyalty_df)

⚠️ inf / -inf 포함 행 개수: 0개


,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate


그니까 지금 anct_id = 17841465816785419 유저는 이전 데이터에는 팔로워가 있다가, 최근에 팔로워가 없어진 케이스라서 follower_loyalty는 계산이 되지만 follower_engagement는 계산이 안되고 있음.
이러한 케이스가 존재할 수 있기 때문에 팔로워가 한명도 없는 유저의 경우 예외처리를 해줘야 함.

콘텐츠 효율성 평가

- 콘텐츠 1개당 팔로워 대비 반응량(like+comments)
- media_agg & media_dtl & acnt_id가 필요 -> 전부 병합한 게 media_engagement_merged_df

In [132]:
# media_engagement_merged_df_3 = media_engagement_profile_merged_df.copy()
# # media_engagment_merged_df_2 == media_engagement_profile_merged_df 같은 df임
# # media_engagement_profile_merged_df_copy = media_engagement_merged_df_2.copy()


# media_engagement_merged_df_3['post_efficiency'] = ((media_engagement_merged_df_3['like_cnt'] + media_engagement_merged_df_3['cmnt_cnt']) / media_engagement_merged_df_3['follower_cnt']) * 100
# post_efficiency_df = media_engagement_merged_df_3.groupby('acnt_id')['post_efficiency'].mean().reset_index()
# post_efficiency_df.rename(columns={'post_efficiency': 'avg_post_efficiency'}, inplace=True)
# post_efficiency_df

In [20]:
def calculate_post_efficiency_df(media_engagement_profile_merged_df):
    media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df.copy()

    media_engagement_profile_merged_df_copy['post_efficiency'] = ((media_engagement_profile_merged_df_copy['like_cnt'] + media_engagement_profile_merged_df_copy['cmnt_cnt']) / media_engagement_profile_merged_df_copy['follower_cnt']) * 100
    post_efficiency_df = media_engagement_profile_merged_df_copy.groupby('acnt_id')['post_efficiency'].mean().reset_index()
    post_efficiency_df.rename(columns={'post_efficiency': 'avg_post_efficiency'}, inplace=True)

    return post_efficiency_df

In [47]:
post_efficiency_df = calculate_post_efficiency_df(media_engagement_profile_merged_df)

In [48]:
check_inf(post_efficiency_df)

⚠️ inf / -inf 포함 행 개수: 1개


,acnt_id,avg_post_efficiency
454,17841465816785419,inf


##### not_connected_influencer_flexmatch_score

In [54]:
# # 크리에이터 활동성
# creator_activity_score = activity_df[['acnt_id', 'activity_score']]
# # 트렌드지수
# creator_trend_score = growth_rate_df[['acnt_id', 'trend_score']]
# # 팔로워 참여도
# follower_engagement = follower_engagment_df[['acnt_id', 'follower_total_engagement']]
# # 팔로워 충성도
# follower_loyalty = follower_loyalty_df[['acnt_id', 'follower_retention_rate']]
# # 콘텐츠 효율성
# post_efficiency = post_efficiency_df[['acnt_id', 'avg_post_efficiency']]

# # data_list
# df_list = [creator_activity_score, creator_trend_score, follower_engagement, follower_loyalty, post_efficiency]

# from functools import reduce

# flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)
# user_info_nm = recent_user_info_mtr[['acnt_id', 'acnt_nm', 'influencer_scale_type']]
# # user_info_nm
# flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')
# flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'activity_score', 'trend_score', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency']]

# flexmatch_score.rename(columns={
#     'avg_upload_interval' : '크리에이터 활동성',
#     'trend_score' : '트렌드지수(팔로워순증가량)', 
#     'follower_total_engagement' : '콘텐츠 참여도' ,
#     'follower_retention_rate' : '팔로워 충성도',
#     'avg_post_efficiency' : '콘텐츠 효율성',
# }, inplace=True)

# flexmatch_score_2 = flexmatch_score.copy()
# flexmatch_score_2.dropna(inplace=True)
# flexmatch_score_2

In [22]:
def not_connected_user_flexmatch_score(activity_df, growth_rate_df, follower_engagement_df, follower_loyalty_df, post_efficiency_df):
    # 크리에이터 활동성
    creator_activity_score = activity_df[['acnt_id', 'activity_score']]
    # 트렌드지수
    creator_trend_score = growth_rate_df[['acnt_id', 'trend_score']]
    # 팔로워 참여도
    follower_engagement = follower_engagement_df[['acnt_id', 'follower_total_engagement']]
    # 팔로워 충성도
    follower_loyalty = follower_loyalty_df[['acnt_id', 'follower_retention_rate']]
    # 콘텐츠 효율성
    post_efficiency = post_efficiency_df[['acnt_id', 'avg_post_efficiency']]

    # data_list
    df_list = [creator_activity_score, creator_trend_score, follower_engagement, follower_loyalty, post_efficiency]

    from functools import reduce

    flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)
    user_info_nm = recent_user_info_mtr[['acnt_id', 'acnt_nm', 'influencer_scale_type']]
    flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')
    flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'influencer_scale_type', 'activity_score', 'trend_score', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency']]

    flexmatch_score.rename(columns={
        'activity_score' : '크리에이터 활동성',
        'trend_score' : '팔로워순증감률', 
        'follower_total_engagement' : '콘텐츠 참여도' ,
        'follower_retention_rate' : '팔로워 충성도',
        'avg_post_efficiency' : '콘텐츠 효율성',
    }, inplace=True)

    not_connected_flexmatch_score_table = flexmatch_score.copy()
    not_connected_flexmatch_score_table.dropna(inplace=True)
    
    return not_connected_flexmatch_score_table

In [23]:
not_connected_flexmatch_score_table = not_connected_user_flexmatch_score(activity_df, growth_rate_df, follower_engagment_df, follower_loyalty_df, post_efficiency_df)
not_connected_flexmatch_score_table

,acnt_id,acnt_nm,influencer_scale_type,크리에이터 활동성,팔로워순증감률,콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성
1,17841400006499942,kim__us_,macro,31.578947,-0.163534,3219.801059,99.84,1.857935
3,17841400016295781,jihyolimm,mid,184.615385,0.000000,603.536863,100.00,1.221735
5,17841400038455668,kos9803,mid,11.940299,-0.020737,290.415000,99.98,0.335352
8,17841400064417438,jjosworlds,mid,141.176471,-0.404967,2484.457250,99.60,1.274734
10,17841400070132367,gnuoyeatt,nano,12.371134,-0.110375,1464.088398,99.89,13.812155
...,...,...,...,...,...,...,...,...
716,17841473332885823,shalala_prc,micro,36.363636,-0.440098,5.721022,99.56,0.220039
717,17841473347851502,hyepick_place,micro,45.283019,-0.882292,109.016651,99.12,3.633888
718,17841473857435791,sesebonbon,micro,200.000000,7.234432,347.076923,100.00,8.676923
719,17841474152637635,_samelee47,nano,28.571429,0.000000,10.000000,100.00,10.000000


In [77]:
not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['acnt_id']=='17841400111322589']

,acnt_id,acnt_nm,influencer_scale_type,크리에이터 활동성,팔로워순증감률,콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성
17,17841400111322589,seon4123,nano,266.666667,-0.206186,1053.487603,99.79,1.024793


In [78]:
not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['acnt_id']=='17841400070132367']

,acnt_id,acnt_nm,influencer_scale_type,크리에이터 활동성,팔로워순증감률,콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성
10,17841400070132367,gnuoyeatt,nano,12.371134,-0.110375,1464.088398,99.89,13.812155


In [24]:
not_connected_flexmatch_score_table['influencer_scale_type'].value_counts()

influencer_scale_type
micro    190
mid      143
nano     114
macro     30
mega       1
Name: count, dtype: int64

In [25]:
nano = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='nano']
micro = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='micro']
mid = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='mid']
macro = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='macro']
mega = not_connected_flexmatch_score_table[not_connected_flexmatch_score_table['influencer_scale_type']=='mega']

In [74]:
# influencer_scale_df_list = [nano, micro, mid, macro, mega]


# for df in influencer_scale_df_list :
#     columns = df.select_dtypes(include='float64').columns
#     cleaned = df[columns].replace([np.inf, -np.inf], np.nan).dropna()
#     minmax = MinMaxScaler(feature_range=(0, 5))
#     flexmatch_score_norm = minmax.fit_transform(cleaned[columns])

#     flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=cleaned.index)
#     flexmatch_score_norm_df['acnt_id'] = df['acnt_id']
#     flexmatch_score_norm_df['acnt_nm'] = df['acnt_nm']

#     flexmatch_score_norm_df['크리에이터 활동성'] = 5 - flexmatch_score_norm_df['크리에이터 활동성']

In [ ]:
influencer_scale_df_list = [nano, micro, mid, macro, mega]
normalized_df_list = []

# 높을수록 안 좋은 지표
reverse_columns = []  

for df in influencer_scale_df_list:
    cleaned = df.copy()

    # 무한대 및 NaN 제거
    float_cols = cleaned.select_dtypes(include='float64').columns
    cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
    cleaned = cleaned.dropna(subset=float_cols)

    if cleaned.empty:
        continue

    # 개별 컬럼 정규화 (0~5 스케일)
    norm_df = pd.DataFrame(index=cleaned.index)
    for col in float_cols:
        scaler = MinMaxScaler(feature_range=(0, 5))
        norm_col = scaler.fit_transform(cleaned[[col]])
        norm_df[col] = 5 - norm_col.ravel() if col in reverse_columns else norm_col.ravel()
        norm_df[col] = norm_col.ravel() if col in reverse_columns else norm_col.ravel()

    # ID 및 이름 붙이기
    norm_df['acnt_id'] = cleaned['acnt_id'].values
    norm_df['acnt_nm'] = cleaned['acnt_nm'].values

    normalized_df_list.append(norm_df)


In [82]:
normalized_df_list[0]

,크리에이터 활동성,팔로워순증감률,콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,acnt_id,acnt_nm
10,0.198536,1.238451,0.900693,4.845938,0.971915,17841400070132367,gnuoyeatt
17,4.443558,1.204167,0.647779,4.705882,0.047514,17841400111322589,seon4123
30,0.777587,1.338594,3.279573,5.000000,0.213176,17841400204233993,j_solha
44,0.219657,1.357639,0.547313,5.000000,0.185162,17841400297953515,j.yyyyyul
70,1.019299,1.402914,2.127321,5.000000,0.154577,17841400483419781,hyeryeon_yoon
...,...,...,...,...,...,...,...
708,3.998404,1.452070,0.131461,5.000000,0.095958,17841469588134780,review.cat
709,0.214598,1.277946,0.004689,5.000000,1.869567,17841470993682107,hi_losy
713,1.244016,5.000000,0.515891,5.000000,1.163201,17841472222414158,elmom.home
719,0.468972,1.277946,0.005033,5.000000,0.696334,17841474152637635,_samelee47


##### 크리에이터 광고 효율성 

In [ ]:
revenue_dic = {
    'acnt_nm' : ['s_h_j_', 'siwolbubu_hyun', 'bong_camper83', 'binwoos', 'seojinii_', 'tingkerhee'],
    'sell_type' : ['flexmatch', 'other', 'flexmatch', 'flexmatch', 'other', 'other'],
    'total_revenue' : [6906000, 10937105, 7233100, 8759000, 7939664, 38449720],
    'total_order_cnt' : [84, 132, 122, 88, 89, 471]
    }

revenue_df = pd.DataFrame(revenue_dic)
revenue_merged_df = pd.merge(media_engagement_merged_df, revenue_df, on='acnt_nm', how='left')
revenue_merged_df = revenue_merged_df[['acnt_id', 'acnt_nm', 'follower_cnt', 'follow_cnt', 'media_cnt', 'sell_type', 'total_revenue', 'total_order_cnt', 'media_id', 'like_cnt', 'cmnt_cnt']]
revenue_merged_df

revenue_merged_df['post_efficiency'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / revenue_merged_df['follower_cnt']) * 100
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'post_efficiency' : 'mean',
    'total_order_cnt' : 'first'
}).dropna()

revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['post_efficiency']) 
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

revenue_merged_df['engagement_per_post'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / 25)
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'engagement_per_post' : 'mean',
    'total_revenue' : 'first',
    'total_order_cnt' : 'first'
}).dropna()

# 팔로워를 생각하지 않고, 25개의 콘텐츠 단위당 반응 비율을 계산
revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['engagement_per_post'] * 25) * 100
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

,acnt_nm,sell_type,total_revenue,total_order_cnt
0,s_h_j_,flexmatch,6906000,84
1,siwolbubu_hyun,other,10937105,132
2,bong_camper83,flexmatch,7233100,122
3,binwoos,flexmatch,8759000,88
4,seojinii_,other,7939664,89
5,tingkerhee,other,38449720,471


콘텐츠 효율성이 아닌 팔로워 참여도를 기준으로 계산했을 때

In [ ]:
engaged_df_2 = engaged_df.copy()
revenue_merged_df_2 = revenue_merged_df.groupby(['acnt_id', 'acnt_nm'])[['total_revenue', 'total_order_cnt']].first().dropna().reset_index()
revenue_df_total_2 = pd.merge(engaged_df_2, revenue_merged_df_2, on='acnt_id')

revenue_df_total_2['advertisement_efficiency'] = revenue_df_total_2['total_order_cnt'] / revenue_df_total_2['follower_total_engagement']
revenue_df_total_2[['acnt_id', 'acnt_nm', 'total_revenue', 'total_order_cnt', 'follower_total_engagement', 'advertisement_efficiency']].sort_values(by='advertisement_efficiency', ascending=False)

table merged

In [583]:
# 크리에이터 활동성
creator_activity_score = activity_df[['acnt_id', 'avg_upload_interval']]

In [584]:
# 트렌드지수
creator_trend_score = time_series_merged_df[['acnt_id', 'trend_score']]

In [585]:
# 팔로워 참여도
follower_engagement = engaged_df[['acnt_id', 'follower_total_engagement']]

In [586]:
# 팔로워 충성도
follower_loyalty = time_series_merged_df_copy[['acnt_id', 'follower_retention_rate']]

In [587]:
# 콘텐츠 효율성
post_efficiency = post_efficiency_df.copy()

In [595]:
# 광고효율성
revenue_df_total = revenue_df_total.reset_index()
advertisement_efficiency = revenue_df_total[['acnt_id', 'advertisement_efficiency']]

In [596]:
df_list = [creator_activity_score, creator_trend_score, follower_engagement, follower_loyalty, post_efficiency, advertisement_efficiency]

In [597]:
from functools import reduce

flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)

In [598]:
user_info_nm = user_info_2[['acnt_id', 'acnt_nm']]
flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')

In [604]:
flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'avg_upload_interval', 'trend_score', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency', 'advertisement_efficiency']]
flexmatch_score.rename(columns={
    'avg_upload_interval' : '크리에이터 활동성',
    'trend_score' : '트렌드지수(팔로워순증가량)', 
    'follower_total_engagement' : '콘텐츠 참여도' ,
    'follower_retention_rate' : '팔로워 충성도',
    'avg_post_efficiency' : '콘텐츠 효율성',
    'advertisement_efficiency' : '광고 효율성'
}, inplace=True)

In [605]:
flexmatch_score_2 = flexmatch_score.copy()
flexmatch_score_2.dropna(inplace=True)

In [606]:
flexmatch_score_2

,acnt_id,acnt_nm,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성
0,17841400361359004,s_h_j_,1.166667,0.057866,110.188904,100.00,0.045178,1859.316062
1,17841400561503844,binwoos,0.500000,-0.014956,626.642600,99.99,0.169730,518.470975
2,17841400591698216,tingkerhee,2.416667,-0.018509,2065.579146,99.98,0.483403,974.343202
4,17841401506106699,siwolbubu_hyun,0.958333,-0.075045,2094.996055,99.92,0.556736,237.096389
6,17841402936102997,seojinii_,2.875000,-0.002312,166.809026,100.00,0.240359,370.279723
10,17841453615191128,bong_camper83,1.500000,0.281793,2526.578680,100.00,2.506526,48.672935


In [608]:
# from sklearn.preprocessing import MinMaxScaler

# columns = flexmatch_score_2.select_dtypes(include='float64').columns
# minmax = MinMaxScaler(feature_range=(0, 5))
# flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

# flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
# flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
# flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

# flexmatch_score_norm_df['avg_upload_interval'] = 5 - flexmatch_score_norm_df['avg_upload_interval']
# flexmatch_score_norm_df


In [ ]:
from sklearn.preprocessing import MinMaxScaler

columns = flexmatch_score_2.select_dtypes(include='float64').columns
minmax = MinMaxScaler(feature_range=(0, 5))
flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

flexmatch_score_norm_df['크리에이터 활동성'] = 5 - flexmatch_score_norm_df['크리에이터 활동성']
flexmatch_score_norm_df

# 지금 이거 정규화 할 때는 그냥 이사람들을 전부 다 합쳐서 함께 정규화를 진행해서 그런데, 원래라면은 팔로워를 기준으로 뭐 메가 인플루언서, 마이크로 인플루언서 이런식으로 기준을 나눠서 정규화를 해야 조금 더 정확한 값이 될 것 같음.


,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성,acnt_id,acnt_nm
0,3.596491,1.862339,0.000000,5.000,0.000000,5.000000,17841400361359004,s_h_j_
1,5.000000,0.841960,1.068647,4.375,0.253016,1.297324,17841400561503844,binwoos
2,0.964912,0.792182,4.046099,3.750,0.890213,2.556192,17841400591698216,tingkerhee
4,4.035088,0.000000,4.106968,0.000,1.039182,0.520322,17841401506106699,siwolbubu_hyun
6,0.000000,1.019133,0.117159,5.000,0.396492,0.888101,17841402936102997,seojinii_
10,2.894737,5.000000,5.000000,5.000,5.000000,0.000000,17841453615191128,bong_camper83


In [ ]:
# flexmatch_score_norm_df.to_csv("flexmatch_score_test.csv")